In [ ]:
!pip install gtfs-kit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 10.0 MB 7.5 MB/s 
     |████████████████████████████████| 1.0 MB 48.8 MB/s 
     |████████████████████████████████| 994 kB 59.9 MB/s 
     |████████████████████████████████| 6.3 MB 56.0 MB/s 
     |████████████████████████████████| 16.7 MB 141 kB/s 
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7609 sha256=fdcd007073878a3d526b1b23bafa25f30f00284005fa3af52f1d2e09be87a47c
  Stored in directory: /root/.cache/pip/wheels/e5/99/37/e1839a5ad733e0d6abb7e0419fd913e8926ddf96408239ce01
  Created wheel for pycountry: filename=pycountry-19.8.18-py2.py3-none-any.whl size=10627377 sha256=3d560fb80742004c9fb3713d2914d9e54103dcc0d3ddf2f24445b17b3e2b9961
  Stored in directory: /root/.cache/pip/wheels/00/77/65/5f318c3c30b046b450a385ecfc690125f8a270cdb0f3f483b4
  Created wheel for u

In [ ]:
!pip install geopandas
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# unzip folder
import time
total_program_time_start = time.time()
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/trimet.gtfs.zip', 'r') as zip_ref:
    zip_ref.extractall("/content/gtfs")

# reading Geojson map data

In [ ]:
import geojson

In [ ]:
import json
path_to_file = "/content/map (2).geojson"
with open(path_to_file) as f:
    gj = geojson.load(f)

full_area = 0
list_of_rectangle_boundaries = []
for i in range(len(gj['features'])):
  print("rectangle "+str(i)+" : ",gj['features'][i]['geometry']['coordinates']) 
  #print(" first point  : ",gj['features'][i]['geometry']['coordinates'][0][0])
  list_of_rectangle_boundaries.append([(gj['features'][i]['geometry']['coordinates'][0][0][0],gj['features'][i]['geometry']['coordinates'][0][0][1]),(gj['features'][i]['geometry']['coordinates'][0][2][0],gj['features'][i]['geometry']['coordinates'][0][2][1])])
  print(list_of_rectangle_boundaries[i])
  print(gj['features'][i]['properties']['area'])
  full_area += float(gj['features'][i]['properties']['area'])


rectangle 0 :  [[[-122.938385, 45.41966], [-122.784576, 45.41966], [-122.784576, 45.572716], [-122.938385, 45.572716], [-122.938385, 45.41966]]]
[(-122.938385, 45.41966), (-122.784576, 45.572716)]
204.49
rectangle 1 :  [[[-122.77771, 45.426407], [-122.674026, 45.426407], [-122.674026, 45.529922], [-122.77771, 45.529922], [-122.77771, 45.426407]]]
[(-122.77771, 45.426407), (-122.674026, 45.529922)]
93.26
rectangle 2 :  [[[-122.720032, 45.532327], [-122.678833, 45.532327], [-122.678833, 45.553006], [-122.720032, 45.553006], [-122.720032, 45.532327]]]
[(-122.720032, 45.532327), (-122.678833, 45.553006)]
	7.39
rectangle 3 :  [[[-122.662354, 45.416286], [-122.55455, 45.416286], [-122.55455, 45.581367], [-122.662354, 45.581367], [-122.662354, 45.416286]]]
[(-122.662354, 45.416286), (-122.55455, 45.581367)]
154.58
rectangle 4 :  [[[-122.54837, 45.41484], [-122.390442, 45.41484], [-122.390442, 45.562622], [-122.54837, 45.562622], [-122.54837, 45.41484]]]
[(-122.54837, 45.41484), (-122.390442, 

In [ ]:
import random
rectangle_to_choose = random.randint(0,len(list_of_rectangle_boundaries)-1)
print(list_of_rectangle_boundaries[rectangle_to_choose])

[(-122.720032, 45.532327), (-122.678833, 45.553006)]


In [ ]:
full_area # in square km

662.47

## Simulation hyperparameters

In [ ]:
number_of_drivers = int(4.3*662.47)
car_speed = 40 #km/h
circuity = 1 # circuity coefficient
detour_rate_percent = 15 # in percent
print(number_of_drivers)

2848


In [ ]:
import math
def compute_distance(stop_ids_list):

  distance = 0

  for i in range(len(stop_ids_list)-1):
    
    stop1 = stop_ids_list[i]
    stop2 = stop_ids_list[i+1]

    lat1 = stops_df[stops_df["stop_id"] == stop1].reset_index()['stop_lat'].loc[0]
    lat2 = stops_df[stops_df["stop_id"] == stop2].reset_index()['stop_lat'].loc[0]
    lon1 = stops_df[stops_df["stop_id"] == stop1].reset_index()['stop_lon'].loc[0]
    lon2 = stops_df[stops_df["stop_id"] == stop2].reset_index()['stop_lon'].loc[0]

    φ1 = lat1 * math.pi/180; # φ, λ in radians
    φ2 = lat2 * math.pi/180;
    Δφ = (lat2-lat1) * math.pi/180;
    Δλ = (lon2-lon1) * math.pi/180;

    p = 0.017453292519943295;    # Math.PI / 180
    c = math.cos;
    a = 0.5 - c((lat2 - lat1) * p)/2 + c(lat1 * p) * c(lat2 * p) * (1 - c((lon2 - lon1) * p))/2;
    distance += 12742 * math.asin(math.sqrt(a)) # 2 * R; R = 6371 km

  return circuity * distance; 

In [ ]:
## get stops as a dataframe
import gtfs_kit as gk

path = '/content/drive/MyDrive/trimet.gtfs.zip'
gk.list_feed(path)
# Read feed and describe
feed = gk.read_feed(path, dist_units='km')
stops_df = feed.stops
stops_df.tail(3)
feed.routes[feed.routes['route_long_name'] == "12th/NE 33rd Ave"].head(5)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color,route_sort_order
61,70,TRIMET,70,12th/NE 33rd Ave,3,https://trimet.org/schedules/r070.htm,NaN,NaN,7100


## Creating stops

In [ ]:
# génerer les origines destinations des passagers
import numpy as np
import pickle

org_dst_dt_list = []
for i in range(number_of_drivers):
  rectangle_to_choose = random.randint(0,len(list_of_rectangle_boundaries)-1)
  rectangle = list_of_rectangle_boundaries[rectangle_to_choose]
  from_stop = (np.random.uniform(rectangle[0][1],rectangle[1][1]),np.random.uniform(rectangle[0][0],rectangle[1][0]))
  rectangle_to_choose = random.randint(0,len(list_of_rectangle_boundaries)-1)
  rectangle = list_of_rectangle_boundaries[rectangle_to_choose]
  to_stop = (np.random.uniform(rectangle[0][1],rectangle[1][1]),np.random.uniform(rectangle[0][0],rectangle[1][0]))
  org_dst_dt_list.append({'from_stop_id' : from_stop,
                          'to_stop_id':to_stop,
                          "departure_time":np.random.randint(10*60+30,11*60+30)})
# save this in pickle file

with open('/content/drive/MyDrive/drivers_org_dst_list.pickle', 'wb') as handle:
      pickle.dump(org_dst_dt_list,handle)

org_dst_dt_list

[{'from_stop_id': (45.56049792815557, -122.9363707681597),
  'to_stop_id': (45.495838083637835, -122.92875332522186),
  'departure_time': 651},
 {'from_stop_id': (45.55234744788274, -122.67976654934547),
  'to_stop_id': (45.470741752806774, -122.80576411341443),
  'departure_time': 660},
 {'from_stop_id': (45.54899779704589, -122.71862249004043),
  'to_stop_id': (45.54047294260969, -122.680382801495),
  'departure_time': 645},
 {'from_stop_id': (45.54468664371471, -122.70195259359194),
  'to_stop_id': (45.49379996150052, -122.5272467787076),
  'departure_time': 656},
 {'from_stop_id': (45.44416080824717, -122.7717239176239),
  'to_stop_id': (45.47075687930115, -122.65907987040724),
  'departure_time': 657},
 {'from_stop_id': (45.4467370342997, -122.84022424865796),
  'to_stop_id': (45.447427108448885, -122.7975726592677),
  'departure_time': 665},
 {'from_stop_id': (45.44212270092694, -122.68590792591134),
  'to_stop_id': (45.5258241835009, -122.46083005006999),
  'departure_time': 671

In [ ]:

# stop_id,stop_code,stop_name,tts_stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,direction,position
def add_stop(lat,lon,org_dst,index):
    stop_id = 'DRIVER_'+org_dst+"_"+str(index)
    stop_name = stop_id
    stop_lon = str(lon)
    stop_lat = str(lat)
    zone_id = "B"
    location_type = "0"
    parent_station = ""#stop_id
    wheelchair_boarding = "0"
    string = stop_id+","+stop_id+","+stop_name+",,,"+stop_lat+","+stop_lon+","+zone_id+",,"+location_type+","+parent_station+",,"
    return string

with open("/content/gtfs/stops.txt", "a") as file_object:
    # Append carpooling route
    i = 1
    for item in org_dst_dt_list:
      
      string = add_stop(item["from_stop_id"][0],item["from_stop_id"][1],"origin",i)
      file_object.write(string)
      file_object.write("\n")
      string = add_stop(item["to_stop_id"][0],item["to_stop_id"][1],"destination",i)
      file_object.write(string)
      i += 1
      file_object.write("\n")

In [ ]:
# folder to zip
import shutil
shutil.make_archive("gtfs", 'zip', "/content/gtfs")

'/content/gtfs.zip'

# Read feed again to include new stops

In [ ]:
# read new feed to include new stops
import gtfs_kit as gk
path = '/content/gtfs'
feed = gk.read_feed(path, dist_units='km')


In [ ]:
# check if stops are correctly added
feed.stops[feed.stops["parent_station"].notnull()].tail(10*number_of_drivers)

,stop_id,stop_code,stop_name,tts_stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,direction,position
4571,8337,8337,Oak/ SW 1st Ave MAX Station,oak/ southwest 1st avenue max station,Northbound stop in Portland (Stop ID 8337),45.520280,-122.672146,R,https://trimet.org/home/stop/8337,0,landmark-138,North,Nearside
4572,8338,8338,Skidmore Fountain MAX Station,skidmore fountain max station,Northbound stop in Portland (Stop ID 8338),45.523095,-122.671256,R,https://trimet.org/home/stop/8338,0,landmark-118,North,At
4573,8339,8339,Old Town/Chinatown MAX Station,"old town, chinatown max station",Northbound stop in Portland (Stop ID 8339),45.525180,-122.671353,R,https://trimet.org/home/stop/8339,0,landmark-110,North,At
4574,8340,8340,Rose Quarter TC MAX Station,rose quarter transit-center max station,Eastbound stop in Portland (Stop ID 8340),45.530057,-122.664917,R,https://trimet.org/home/stop/8340,0,landmark-137,East,At
4575,8341,8341,Convention Center MAX Station,convention center max station,Eastbound stop in Portland (Stop ID 8341),45.530052,-122.661908,R,https://trimet.org/home/stop/8341,0,landmark-119,East,At
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6401,13725,13725,SE 17th Ave & Rhine St MAX Station,southeast 17th avenue & rhine street max station,Northbound stop in Portland (Stop ID 13725),45.498131,-122.648142,R,https://trimet.org/home/stop/13725,0,landmark-2555,North,At
6402,13726,13726,Clinton St/SE 12th Ave MAX Station,"clinton street, southeast 12th avenue max station",Westbound stop in Portland (Stop ID 13726),45.502953,-122.653516,R,https://trimet.org/home/stop/13726,0,landmark-2554,West,At
6403,13727,13727,OMSI/SE Water MAX Station,"omsi, southeast water max station",Westbound stop in Portland (Stop ID 13727),45.506618,-122.662862,R,https://trimet.org/home/stop/13727,0,landmark-2553,West,At
6404,13728,13728,South Waterfront/S Moody MAX Station,"south waterfront, south moody max station",Westbound stop in Portland (Stop ID 13728),45.502716,-122.671539,R,https://trimet.org/home/stop/13728,0,landmark-2552,West,At


## Get stops that are train stations

In [ ]:
stops_df = feed.stops
stations_df = stops_df[stops_df["location_type"] == 1]
len(stations_df)


69

## Get stations that have more than one train route

In [ ]:
import pandas as pd
def number_of_routes(station_id):
  stops_df = feed.stops
  stations_df = stops_df
  station_df = stations_df[stations_df['parent_station'] == station_id]
  station_df
  stop_times_df = feed.stop_times
  df_station_stops = pd.merge(station_df,stop_times_df,on='stop_id')[['stop_name','trip_id', 'arrival_time', 'departure_time', 'stop_id']]
  df_station_stops
  trips_df = feed.trips
  df_station_trips = pd.merge(df_station_stops,trips_df[['trip_id', 'route_id', 'service_id']],on='trip_id',how='left')
  df_station_trips.sort_values(by="arrival_time")
  routes_df = feed.routes
  df_station_trips_route = pd.merge(df_station_trips,routes_df[['route_id','route_short_name','route_type']],on="route_id",how='left')
  # On obtient la colonne avec les lignes de métro 1 et 5 aussi
  df_station_trips_route
  #print(df_station_trips_route['route_id'].unique())
  return len(df_station_trips_route['route_id'].unique())

In [ ]:
stations_df["number_of_routes"] = stations_df['stop_id'].apply(lambda x : number_of_routes(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
stations_df

,stop_id,stop_code,stop_name,tts_stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,direction,position,number_of_routes
6698,landmark-89,NaN,Albina/Mississippi MAX Station,"al-beina, mississippi max station",NaN,45.539555,-122.675497,NaN,NaN,1,NaN,NaN,NaN,1
6699,landmark-97,NaN,Beaverton Central MAX Station,beaverton central max station,NaN,45.490463,-122.807104,NaN,NaN,1,NaN,NaN,NaN,1
6700,landmark-95,NaN,Beaverton Creek MAX Station,beaverton creek max station,NaN,45.500417,-122.833055,NaN,NaN,1,NaN,NaN,NaN,1
6701,landmark-96,NaN,Beaverton TC MAX Station,beaverton transit-center max station,NaN,45.491513,-122.801462,NaN,NaN,1,NaN,NaN,NaN,2
6702,landmark-111,NaN,Cascades MAX Station,cascades max station,NaN,45.572233,-122.557995,NaN,NaN,1,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6762,landmark-2563,NaN,Tigard TC WES Station,tigard transit-center wes station,NaN,45.430084,-122.769663,NaN,NaN,1,NaN,NaN,NaN,1
6763,landmark-107,NaN,Tuality Hospital/SE 8th Ave MAX Station,"tuality hospital, southeast 8th avenue max sta...",NaN,45.521301,-122.978373,NaN,NaN,1,NaN,NaN,NaN,1
6764,landmark-113,NaN,Washington Park MAX Station,washington park max station,NaN,45.510657,-122.716792,NaN,NaN,1,NaN,NaN,NaN,2
6765,landmark-106,NaN,Washington/SE 12th Ave MAX Station,"washington, southeast 12th avenue max station",NaN,45.521166,-122.969510,NaN,NaN,1,NaN,NaN,NaN,1


In [ ]:
stations_df = stations_df[stations_df["number_of_routes"] > 1]
len(stations_df)

16

## Create detours
We want drivers to have detours to the closest station

In [ ]:
# get closest station to origin
from tqdm import tqdm
import time
start = time.time()

stations_closer_to_origin = []
stations_closer_to_destination = []
'''
for i in tqdm(range(number_of_drivers)):

  stop_to_compare_to = "DRIVER_origin_"+str(i+1)  
  stations_df["distance"] = [compute_distance([str(stop_to_compare_to),str(x)]) for x in stations_df["stop_id"]]
  df = stations_df.sort_values(by = "distance")
  #print("the closest station to "+stop_to_compare_to+" is : "+df["stop_name"].iloc[0]+" distance : ",df["distance"].iloc[0])
  # get the closest stop with stop type 0 with this station as parent
  closest_station_id = feed.stops[feed.stops["parent_station"] == df["stop_id"].iloc[0]]["stop_id"].iloc[0]
  stations_closer_to_origin.append(closest_station_id) # add the stop_id to list

  stop_to_compare_to = "DRIVER_destination_"+str(i+1)  
  stations_df["distance"] = [compute_distance([str(stop_to_compare_to),str(x)]) for x in stations_df["stop_id"]]
  df = stations_df.sort_values(by = "distance")
  #print("the closest station to "+stop_to_compare_to+" is : "+df["stop_name"].iloc[0]+" distance : ",df["distance"].iloc[0])
  # get the closest stop with stop type 0 with this station as parent
  closest_station_id = feed.stops[feed.stops["parent_station"] == df["stop_id"].iloc[0]]["stop_id"].iloc[0]
  stations_closer_to_destination.append(closest_station_id) # add the stop_id to list
'''
print("this took about : ",time.time() - start)
print("closest stations to origins list = ",stations_closer_to_origin)
print("closest stations to destinations list = ",stations_closer_to_destination)

this took about :  9.965896606445312e-05
closest stations to origins list =  []
closest stations to destinations list =  []


## Create trajectories

In [ ]:
def get_trajectories_list(number_of_drivers,stations_closer_to_origin,stations_closer_to_destination):
  
  trajectories_list = []

  for i in tqdm(range(number_of_drivers)):

    trajectory = ['DRIVER_origin_'+str(i+1),'DRIVER_destination_'+str(i+1)]
    
    prob = np.random.randint(0,1)

    if prob == 0 :
      stop_to_compare_to = "DRIVER_origin_"+str(i+1)  
      stations_df["distance"] = [compute_distance([str(stop_to_compare_to),str(x)]) for x in stations_df["stop_id"]]
      df = stations_df.sort_values(by = "distance")
      #print("the closest station to "+stop_to_compare_to+" is : "+df["stop_name"].iloc[0]+" distance : ",df["distance"].iloc[0])
      # get the closest stop with stop type 0 with this station as parent
      closest_station_id = feed.stops[feed.stops["parent_station"] == df["stop_id"].iloc[0]]["stop_id"].iloc[0]
      stations_closer_to_origin.append(closest_station_id)
      
      original_distance = compute_distance(trajectory)

      detour_distance = compute_distance([trajectory[0],stations_closer_to_origin[i],trajectory[1]])
      
      detour_percentage = (detour_distance - original_distance)*100/original_distance
      
      #print(detour_percentage)

      if detour_percentage < detour_rate_percent :
        
        trajectories_list.append([trajectory[0],stations_closer_to_origin[i],trajectory[1]])
      
      else :
        trajectories_list.append(trajectory)
    
    if prob == 1 :

      stop_to_compare_to = "DRIVER_destination_"+str(i+1)  
      stations_df["distance"] = [compute_distance([str(stop_to_compare_to),str(x)]) for x in stations_df["stop_id"]]
      df = stations_df.sort_values(by = "distance")
      #print("the closest station to "+stop_to_compare_to+" is : "+df["stop_name"].iloc[0]+" distance : ",df["distance"].iloc[0])
      # get the closest stop with stop type 0 with this station as parent
      closest_station_id = feed.stops[feed.stops["parent_station"] == df["stop_id"].iloc[0]]["stop_id"].iloc[0]
      stations_closer_to_destination.append(closest_station_id) # add the stop_id to list

      original_distance = compute_distance(trajectory)

      detour_distance = compute_distance([trajectory[0],stations_closer_to_destination[i],trajectory[1]])
      
      detour_percentage = (detour_distance - original_distance)*100/original_distance
      
      #print(detour_percentage)

      if detour_percentage < detour_rate_percent :
        
        trajectories_list.append([trajectory[0],stations_closer_to_destination[i],trajectory[1]])
      
      else :

        trajectories_list.append(trajectory)

  return trajectories_list

trajectories_list = get_trajectories_list(number_of_drivers,stations_closer_to_origin,stations_closer_to_destination)
trajectories_list  

  0%|          | 0/2848 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
100%|██████████| 2848/2848 [07:19<00:00,  6.48it/s]


[['DRIVER_origin_1', 'DRIVER_destination_1'],
 ['DRIVER_origin_2', 'DRIVER_destination_2'],
 ['DRIVER_origin_3', 'DRIVER_destination_3'],
 ['DRIVER_origin_4', '9757', 'DRIVER_destination_4'],
 ['DRIVER_origin_5', 'DRIVER_destination_5'],
 ['DRIVER_origin_6', 'DRIVER_destination_6'],
 ['DRIVER_origin_7', 'DRIVER_destination_7'],
 ['DRIVER_origin_8', '9818', 'DRIVER_destination_8'],
 ['DRIVER_origin_9', 'DRIVER_destination_9'],
 ['DRIVER_origin_10', '9624', 'DRIVER_destination_10'],
 ['DRIVER_origin_11', '10120', 'DRIVER_destination_11'],
 ['DRIVER_origin_12', 'DRIVER_destination_12'],
 ['DRIVER_origin_13', '10120', 'DRIVER_destination_13'],
 ['DRIVER_origin_14', 'DRIVER_destination_14'],
 ['DRIVER_origin_15', 'DRIVER_destination_15'],
 ['DRIVER_origin_16', 'DRIVER_destination_16'],
 ['DRIVER_origin_17', 'DRIVER_destination_17'],
 ['DRIVER_origin_18', 'DRIVER_destination_18'],
 ['DRIVER_origin_19', 'DRIVER_destination_19'],
 ['DRIVER_origin_20', '9624', 'DRIVER_destination_20'],
 ['DRIVE

In [ ]:
#trajectories_list = [['DRIVER_origin_1', '5319', 'DRIVER_destination_1'],['DRIVER_origin_2', '5319', 'DRIVER_destination_2']]

## Functions for creating lines in GTFS files

In [ ]:
import time
def minutes_to_date(mins):
  formatted_time = time.strftime('%H:%M:%S', time.gmtime(mins*60))
  return str(formatted_time)

def create_carpooling_routes(number_of_routes):

  strings_list = []

  for i in range(number_of_routes):
      route_id = "100"+str(i)
      route_short_name = route_id
      route_long_name = "route of carpooler number "+str(i+1)

      # columns = [route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color,route_sort_order]
      strings_list.append(route_id+",TRIMET,"+route_short_name+","+route_long_name+",3,https://trimet.org/schedules/r012.htm,,,1500")
                          
  return route_id, strings_list

def create_carpooling_trips(number_of_trips):
  
  strings_list = []
  
  for i in range(number_of_trips):
    route_id = "100"+str(i)
    trip_id = "1162238700"+str(i)
    #string = route_id+","+service_id+","+trip_id+","+trip_headsign+","+direction_id+","
    strings_list.append(route_id+",W.625,"+trip_id+",1,1236,512023,,1")
  
  return trip_id,strings_list

def create_carpooling_stop_times_for_single_paths(trajectories_list,org_dst_dt_list):
  # copied GTFS for the bus line I wanted to replicate and changed the tripID and arrival time
  # trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,continuous_drop_off,continuous_pickup
  driver_count = 0
  strings_list = []
  
  for trajectory in trajectories_list :
    driver_count += 1
    travel_time_in_minutes = 0
    for i in range(len(trajectory)):
      if i == 0 :
        travel_time_in_minutes = org_dst_dt_list[driver_count-1]["departure_time"]
        shape_dist_traveled = 0
      else : 
        distance = compute_distance([trajectory[i-1],trajectory[i]])
        travel_time_in_minutes += distance*60/car_speed
        shape_dist_traveled += distance*1000
      trip_id =  "1162238700"+str(driver_count-1)
      formatted_travel_time = minutes_to_date(travel_time_in_minutes)
      stop_id = trajectory[i]
      stop_order = str(24 + i)
         
      strings_list.append(trip_id+","+formatted_travel_time+","+formatted_travel_time+","+stop_id+","+stop_order+",carpooling stop,0,0,"+str(shape_dist_traveled)+",0,,")
  '''
  strings_list = ["2111622387,18:24:01,18:24:01,DRIVER_origin_1,24,Parkrose TC via Portland City Ctr,0,0,0,0,,",
                  "2111622387,18:50:01,18:50:01,5319,25,Parkrose TC via Portland City Ctr,1,0,50.5,0,,",
                  "2111622387,19:00:00,19:00:00,DRIVER_destination_1,79,"We're Hiring - $7,500 Bonus",1,0,82677.0,1,,"]
  '''
  return strings_list



In [ ]:
# this one is for writing in the files
# add carpooling routes
routes = []
# Open a file with access mode 'a'
with open("/content/gtfs/routes.txt", "a") as file_object:
      number_of_routes = number_of_drivers
      route_id,strings_list = create_carpooling_routes(number_of_routes)
      routes.append(route_id)
      for item in strings_list:
        
        file_object.write(item)
        file_object.write("\n")

# add carpooling trips
trips = []
# Open a file with access mode 'a'
with open("/content/gtfs/trips.txt", "a") as file_object:
      number_of_trips = number_of_drivers
      trip_id, strings_list = create_carpooling_trips(number_of_trips)
      trips.append(trip_id)
      for item in strings_list:
        
        file_object.write(item)
        file_object.write("\n")


with open("/content/gtfs/stop_times.txt", "a") as file_object:
    strings_list = create_carpooling_stop_times_for_single_paths(trajectories_list,org_dst_dt_list)
    #all_carpoolers_stops.append(stops_list)
    for item in strings_list:
      
      file_object.write(item)
      file_object.write("\n")



## Making a zip out of everything

In [ ]:
import shutil
shutil.make_archive("/content/drive/MyDrive/walking-driving.gtfs", 'zip', "/content/gtfs")

'/content/drive/MyDrive/walking-driving.gtfs.zip'